In [ ]:
!nvidia-smi

Fri Oct 25 02:23:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Get Images + Metadata

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

ValueError: mount failed

In [ ]:
%cd drive/MyDrive/T4SG St Jude/Labelled Assets/
!ls


In [ ]:
! pip install transformers
! pip install torch

In [ ]:
! pip install exifread

import os
import exifread
from tqdm import tqdm

sample_images = []

missing_count = 0
for filepath_i in tqdm(os.listdir(os.getcwd())):
  # Open image file for reading (must be in binary mode)
  f = open(filepath_i, 'rb')

  # Return Exif tags
  tags = exifread.process_file(f)

  try:
    actual_labels = {
        "description" : tags["Image ImageDescription"]
    }

    sample_images.append((filepath_i, actual_labels))
  except:
    missing_count += 1

print(f"Recovered {len(sample_images)}, Missing {missing_count}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.9 MB/s eta 0:00:00


100%|██████████| 199/199 [00:03<00:00, 61.89it/s] 

Recovered 175, Missing 24


# Try Image-Captioning Models

There are a variety of pre-trained models which take in an image and a text-prompt and output a caption. We'll try seeing how well these do on our sample images without any fine-tuning.

- [Paligemma](https://huggingface.co/google/paligemma-3b-pt-448)
- [BLIP](https://github.com/salesforce/BLIP?tab=readme-ov-file)


[There's a lot of metadata we hope to predict, but the sample images don't currently have those.](https://docs.google.com/spreadsheets/d/1TWElrWaR4sm3CcFMju4zlzgFmKe5Uem1/edit?usp=sharing&ouid=115752792879348590018&rtpof=true&sd=true).

Let's start by predicting the "description," which looks to be a list of words describing the image, and "categories," which is not in the sample images but we can try first.

In [ ]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from huggingface_hub import notebook_login
from PIL import Image
import requests
import torch

model_id = "google/paligemma-3b-mix-224"

notebook_login() # The code won't work at first since you need to do this

model = PaliGemmaForConditionalGeneration.from_pretrained(model_id).eval()
processor = AutoProcessor.from_pretrained(model_id)

output = []

for filepath_i, actual_i in sample_images:
  # Read filepath_i, actual_i
  print(filepath_i, actual_i)

  # TODO: PROCESS THE IMAGE AND SEE THE DESCRIPTION
  image = Image.open(filepath_i)
  prompt = "gene en"
  model_inputs = processor(text=prompt, images=image, return_tensors="pt")
  input_len = model_inputs["input_ids"].shape[-1]

  with torch.inference_mode():
    generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]
    decoded = processor.decode(generation, skip_special_tokens=True)
    print(decoded)

  # STORE THE IMAGE, THE ACTUAL DESCRIPTION, AND THE PREDICTED DESCRIPTION
  output.append({
      "image_path" : filepath_i,
      "actual": actual_i,
      "predicted" : decoded, #TODO ADD SOMETHING HERE
  })

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

KeyboardInterrupt: 

## Output Results in a CSV

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(output)
df.to_csv("Huang_Sam_Paligemma_TokenTest.csv")